In [2]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS

### Forward selection

In [84]:
numrows = 1000
numcols = 10
np.random.seed(0)
X = np.random.normal(0, 1, (numrows, numcols))
coefs = np.random.uniform(1, 5, (numcols,))
Y = X @ coefs + np.random.normal(0, 1, (numrows,))

In [85]:
coefs

array([1.38266659, 2.60768308, 3.10220095, 3.9293287 , 1.5411149 ,
       2.76092265, 3.78260759, 3.53155077, 1.3361475 , 2.34574745])

In [86]:
cols_used = list()
rsq_obtained = list()
cols_unused = np.arange(numcols)

In [89]:
cols_used.append(1)

In [90]:
cols_used

[1]

In [91]:
cols_used = list()
rsq_obtained = list()
cols_unused = np.arange(numcols)
for n in range(numcols):
    cur_col = ""
    cur_rsq = -100
    for col in cols_unused:
        cols_used_extended = list(set(cols_used).union({col}))
        X_next = X[:, cols_used_extended]
        results = sm.OLS(Y, X_next).fit()
        rsq = results.rsquared
        if rsq > cur_rsq:
            cur_col = col
            cur_rsq = rsq
    cols_unused = list(set(cols_unused) - {cur_col})
    cols_used.append(cur_col)
    rsq_obtained.append(cur_rsq)
print(cols_used)
print([f"{x:.3}" for x in rsq_obtained])

[3, 6, 7, 2, 5, 1, 9, 4, 0, 8]
['0.175', '0.35', '0.531', '0.655', '0.741', '0.829', '0.9', '0.931', '0.959', '0.986']


In [92]:
results = sm.OLS(Y, X[:, [3, 6, 7, 2]]).fit()
results.rsquared

0.6549564433469119

### Backward selection

In [93]:
cols_unused = list()
rsq_obtained = list()
cols_used = np.arange(numcols)
for n in range(numcols - 1):
    cur_col = ""
    cur_rsq = -100
    for col in cols_used:
        cols_used_reduced = list(set(cols_used) - {col})
        X_next = X[:, cols_used_reduced]
        results = sm.OLS(Y, X_next).fit()
        rsq = results.rsquared
        if rsq > cur_rsq:
            cur_col = col
            cur_rsq = rsq
    cols_used = list(set(cols_used) - {cur_col})
    cols_unused.append(cur_col)
    rsq_obtained.append(cur_rsq)
print(cols_unused)
print([f"{x:.3}" for x in rsq_obtained])

[8, 0, 4, 9, 1, 5, 2, 7, 6]
['0.959', '0.931', '0.9', '0.829', '0.741', '0.655', '0.531', '0.35', '0.175']


### Lasso and Ridge Regression

In [94]:
numrows = 1000
numcols = 10
X = np.random.normal(0, 1, (numrows, numcols))

In [95]:
coefs = np.hstack((np.zeros(int(numcols / 2),), np.random.uniform(1, 5, (int(numcols / 2),))))
coefs

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       1.2232724 , 4.02698937, 3.98064501, 2.19661032, 1.7423786 ])

In [96]:
Y = X @ coefs + np.random.normal(0, 1, (numrows,))

In [97]:
df = pd.DataFrame(X, columns = list(range(10)))

### Lasso

In [107]:
results = OLS(Y, df).fit_regularized(method = 'elastic_net', alpha = 1.0, L1_wt = 1.0)

In [108]:
results.params

0    0.000000
1    0.000000
2    0.000000
3    0.000000
4    0.000000
5    0.243532
6    3.018165
7    3.007673
8    1.155491
9    0.714582
dtype: float64

In [109]:
root_mean_squared_error(Y, X @ results.params)

2.443132405756472

### Null model - mean only

In [110]:
np.std(Y - Y.mean()) # null model - just predict the mean

6.495337013481665

### Refit model - assume Lasso's chosen coefs are all that matters

In [111]:
X2 = df.iloc[:, np.argwhere(results.params > 0).ravel()]
results2 = OLS(Y, X2).fit()

In [112]:
results2.params

5    1.248863
6    4.037761
7    3.964845
8    2.177846
9    1.709469
dtype: float64

In [113]:
results2.rsquared

0.9771330313647766

In [114]:
root_mean_squared_error(Y, X2 @ results2.params)

0.9846601241494901

### Ridge

In [115]:
results3 = OLS(Y, X).fit_regularized(method = 'elastic_net', alpha = 0.1, L1_wt = 0.0)

In [116]:
results3.params

array([ 1.06918540e-02, -6.31073932e-03, -9.14902410e-03, -2.97657118e-03,
       -1.67093895e-02,  1.13856709e+00,  3.67151430e+00,  3.60913425e+00,
        1.96926813e+00,  1.55871222e+00])

In [61]:
root_mean_squared_error(Y, X @ results3.params)

1.169149416426918

### Test data

In [73]:
X_test = np.random.normal(0, 1, (numrows, numcols))
Y_test = X_test @ coefs + np.random.normal(0, 1, (numrows,))

In [74]:
root_mean_squared_error(Y_test, X_test @ results.params)

2.2712726717197023

In [75]:
df_test = pd.DataFrame(X_test, columns = list(range(10)))
X_test_cut = df_test.iloc[:, np.argwhere(coefs > 0).ravel()]
root_mean_squared_error(Y_test, X_test_cut @ results2.params)

1.016817858147878

In [76]:
root_mean_squared_error(Y_test, X_test @ results3.params)

1.1492647139780656